In [1]:
import pandas as pd
import os
import requests 


Downloading small subset of images - 1000

In [2]:
df = pd.read_csv("Picture Stuff/multimedia.txt", sep="\t")
df = df.dropna(subset=["identifier"])  # Drop rows without image URLs
img_urls = df["identifier"].tolist()

# Create folder, does nothing if it already exists
img_folder = "images" 
os.makedirs(img_folder, exist_ok=True)

for i, url in enumerate(img_urls[3000:8000], start=3001):
    try:
        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            file_path = os.path.join(img_folder, f"image_{i}.jpg")
            with open(file_path, "wb") as f:
                f.write(response.content)

            print(f"Downloading image {i}")
        else:
            print(f"Skipped image {i} (bad response)")
    
    except Exception as e:
        print(f"Error downloading image {i} from {url}: {e}")

Error downloading image 3901 from https://plant.depo.msu.ru/open/public/item/MW0469238-2/img/0.jpg?gbif: HTTPSConnectionPool(host='plant.depo.msu.ru', port=443): Read timed out. (read timeout=10)
Skipped image 4213 (bad response)
Skipped image 4214 (bad response)
Error downloading image 4336 from https://id.digitarium.fi/api/Q.495225/Preview001.jpg: HTTPSConnectionPool(host='id.digitarium.fi', port=443): Max retries exceeded with url: /api/Q.495225/Preview001.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C12FCB8E50>, 'Connection to id.digitarium.fi timed out. (connect timeout=10)'))
Error downloading image 4337 from https://id.digitarium.fi/api/Q.495227/Preview001.jpg: HTTPSConnectionPool(host='id.digitarium.fi', port=443): Max retries exceeded with url: /api/Q.495227/Preview001.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C110967A60>, 'Connection to id.digitarium.fi timed out. (connect timeout=10)')

In [65]:
#For Grabbing all metadata information of pictures
df = pd.read_csv("Picture Stuff/multimedia.txt", sep="\t")
df = df.dropna(subset=["identifier"])  # Drop rows without image URLs
img_urls = df["identifier"].tolist()

img_num = 37

print(df.iloc[img_num-1].to_frame().T)
print(img_urls[img_num-1])  


       gbifID        type      format  \
36  694468213  StillImage  image/jpeg   

                                           identifier references title  \
36  http://mediaphoto.mnhn.fr/media/1441420089062y...        NaN   NaN   

   description source audience created creator contributor publisher  \
36         NaN    NaN      NaN     NaN     NaN         NaN       NaN   

                                        license rightsHolder  
36  http://creativecommons.org/licenses/by/4.0/          NaN  
http://mediaphoto.mnhn.fr/media/1441420089062yjXgSrHdTvSHeDb3


In [1]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image

# Model & processor
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-large")
model = AutoModel.from_pretrained("facebook/dinov2-large")
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# List of image file paths
image_paths = [
    "images/image_1.jpg",
    "images/image_2.jpg",
    "images/image_3.jpg",
    "images/image_4.jpg"
]

embeddings_list = []

for img_path in image_paths:
    image = Image.open(img_path).convert("RGB")  # Make sure image is RGB

    # Preprocess image
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract last hidden state (shape: [batch_size, seq_len, 1024])
    last_hidden_state = outputs.last_hidden_state

    # Take the [CLS] token embedding (index 0) as the image embedding
    # Result is a tensor of shape [1, 1024]
    embedding = last_hidden_state[:, 0, :]

    # Convert to 1D numpy array and append
    embeddings_list.append(embedding.cpu().numpy().flatten())

print(f"Extracted embeddings shape for {len(embeddings_list)} images:")
for i, emb in enumerate(embeddings_list):
    print(f"Image {i+1}: embedding shape {emb.shape}")


c:\Users\abdul\miniconda3\envs\dinov2-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
c:\Users\abdul\miniconda3\envs\dinov2-env\lib\site-packages\transformers\integrations\sdpa_attention.py:66: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Extracted embeddings shape for 4 images:
Image 1: embedding shape (1024,)
Image 2: embedding shape (1024,)
Image 3: embedding shape (1024,)
Image 4: embedding shape (1024,)
